In [1]:
# importation de bibliothèques
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import dagshub
dagshub.init(repo_owner='abdoulaye062012', repo_name='Ml_flow_dagshub_Projet_IA_2025', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()

Accessing as abdoulaye062012

Initialized MLflow to track repo "abdoulaye062012/Ml_flow_dagshub_Projet_IA_2025"

Repository abdoulaye062012/Ml_flow_dagshub_Projet_IA_2025 initialized!

In [3]:
import mlflow
mlflow.set_tracking_uri(uri="https://dagshub.com/abdoulaye062012/Ml_flow_dagshub_Projet_IA_2025.mlflow")

In [2]:
import chardet

with open("zomato.csv", "rb") as f:
    print(chardet.detect(f.read(100000)))

{'encoding': 'MacRoman', 'confidence': 0.707087591240876, 'language': ''}


In [3]:
# telechargement des données
df = pd.read_csv("zomato.csv",delimiter=',',encoding='MacRoman')
# affichage des 5 premières lignes du dataframe
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [4]:
# liste des colonnes
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

Dans notre projet de systeme de recommandation de restaurant nous n'avons pas besoin de toutes ces colonnes 
On a juste besoin de ['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Latitude', 'Longitude', 'Aggregate rating']
Car nous voulons Recommandation basée sur le contenu (type de cuisine, budget, localisation)

In [5]:
new_df = df[['Restaurant Name',"Country Code", 'Cuisines', 'Average Cost for two', 'Latitude', 'Longitude', 'Aggregate rating']]
new_df.head()

,Restaurant Name,Country Code,Cuisines,Average Cost for two,Latitude,Longitude,Aggregate rating
0,Le Petit Souffle,162,"French, Japanese, Desserts",1100,14.565443,121.027535,4.8
1,Izakaya Kikufuji,162,Japanese,1200,14.553708,121.014101,4.5
2,Heat - Edsa Shangri-La,162,"Seafood, Asian, Filipino, Indian",4000,14.581404,121.056831,4.4
3,Ooma,162,"Japanese, Sushi",1500,14.585318,121.056475,4.9
4,Sambo Kojin,162,"Japanese, Korean",1500,14.584450,121.057508,4.8


Nous allons present faire notre systeme de recommandation

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
new_df.dropna(inplace=True)
new_df.reset_index(drop=True, inplace=True)
# me retourne un tableau ligne = nbr restaurants, colonne = nbr cuisines
# avec des 1 et des 0
new_df["Cuisines"] = new_df["Cuisines"].str.lower().str.strip()


C:\Users\Abdallah_Diagne\AppData\Local\Temp\ipykernel_16564\2285885252.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)
C:\Users\Abdallah_Diagne\AppData\Local\Temp\ipykernel_16564\2285885252.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Cuisines"] = new_df["Cuisines"].str.lower().str.strip()


On creer un vecteur qui apprend toutes les cuisines et ensuite donner comme resultats un tableau ou chaque ligne est un restaurant et on donne si vrai ou faux la cuisine est fournie dans ce restaurant. Donc chaque colonne est un restaurant

In [7]:
# encodage des cuisines
from sklearn.feature_extraction.text import CountVectorizer

# Encodage des cuisines
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
cuisine_matrix = vectorizer.fit_transform(new_df['Cuisines'])

c:\Users\Abdallah_Diagne\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Resultat de notre vectorisation

In [8]:
pd.DataFrame(
    cuisine_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

,afghani,african,american,andhra,arabian,argentine,armenian,asian,asian fusion,assamese,...,teriyaki,tex-mex,thai,tibetan,turkish,turkish pizza,vegetarian,vietnamese,western,world cuisine
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


On normalise le bugdet pour mettre les elements a la meme echelle

In [9]:
scaler = MinMaxScaler()
budget_scaled = scaler.fit_transform(new_df[['Average Cost for two']])
# Transformer en vecteur colonne
budget_scaled = np.array(budget_scaled).reshape(-1,1)

On utilise la latitudes et la longitude pour avoir la distances entre les restaurant

In [48]:
# Similarité geographique
from sklearn.metrics.pairwise import haversine_distances
import numpy as np

# Convertir lat/lon en radians
coords = np.radians(new_df[['Latitude', 'Longitude']])

# Distance géographique entre restaurants
geo_dist = haversine_distances(coords) * 6371  # en km

Visualisation des distances

In [63]:
dist_entre_restau = pd.DataFrame(
    geo_dist,
    columns=new_df.index,  # indices uniques
    index=new_df.index
)
dist_entre_restau.loc[0]

0          0.000000
1          1.947587
2          3.617973
3          3.818861
4          3.856326
           ...     
9537    9119.466797
9538    9114.538000
9539    9113.990283
9540    9113.825930
9541    9116.387305
Name: 0, Length: 9542, dtype: float64

Nous allons utiliser notre propre position pour l'integrer dans notre systemes . Cette fonctionnalité vient d'etre ajouter pour permetre au client de choisir le bon restaurant en fonction de la distance

In [21]:
import geocoder

# Localisation à partir de ton IP publique
g = geocoder.ip('me')
print("Adresse IP :", g.ip)
print("Ville :", g.city)
print("Pays :", g.country)
print("Latitude :", g.latlng[0])
print("Longitude :", g.latlng[1])

Adresse IP : 70.51.143.74
Ville : Gatineau
Pays : CA
Latitude : 45.4772
Longitude : -75.7016


In [11]:
# concatenation cuisines_matrix et budget_scaled
from scipy.sparse import hstack

# Fusion cuisines + budget
content_matrix = hstack([cuisine_matrix, budget_scaled])

In [12]:
# matrice de similarité cosinus
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(content_matrix, content_matrix)

code pour calculer la distance entre la position du client et le restaurant 

In [71]:
import requests
from math import radians, sin, cos, sqrt, atan2

def distance_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# retourner une serie avec la distance entre chaque restaurant et la position donnée
def distances_from_location(lat, lon, new_df):
    distances = []
    for i, row in new_df.iterrows():
        dist = distance_km(lat, lon, row['Latitude'], row['Longitude'])
        distances.append(dist)
    return pd.Series(distances)

In [80]:
# fonction de recommandation
def recommandation(restaurant_name,n=5,alpha = 0.7):
    if restaurant_name not in new_df['Restaurant Name'].values:
        return "Restaurant non trouvé"
    idx = new_df[new_df['Restaurant Name'] == restaurant_name].index[0]
    score = list(enumerate(similarity[idx]))
    score = sorted(score, key=lambda x: x[1], reverse=True)
    # score = score[1:n+1]
    restaurant_indices = [i[0] for i in score]
    sim_values = [i[1] for i in score]
    recommandations = new_df[['Restaurant Name', "Country Code","Latitude","Longitude"]].iloc[restaurant_indices]
    recommandations["Similarité"] = sim_values
    recommandations["Distance du client (km)"] = recommandations.apply(
        lambda row: distance_km(g.latlng[0], g.latlng[1], row["Latitude"], row["Longitude"]),
        axis=1
    )
    max_dist = recommandations["Distance du client (km)"].max() or 1
    distance_score = 1 - (recommandations["Distance du client (km)"] / max_dist)
    # Score combiné : 70% similarité + 30% proximité
    recommandations["Score combiné"] = alpha * recommandations["Similarité"] + (1 - alpha) * distance_score
    distances = dist_entre_restau.loc[idx, restaurant_indices]
    result = recommandations.copy()
    recommandations = recommandations.sort_values(by="Score combiné", ascending=False).head(n).reset_index(drop=True)
    return recommandations
# exemple d'utilisation

a = recommandation("Pizza Hut",n=10)

In [81]:
a

,Restaurant Name,Country Code,Latitude,Longitude,Similarité,Distance du client (km),Score combiné
0,Santos,215,53.513525,-2.246548,1.0,5138.808076,0.916126
1,Delhi Pizza Corner,1,0.000000,0.000000,1.0,8898.655036,0.854759
2,Domino's Pizza,1,0.000000,0.000000,1.0,8898.655036,0.854759
3,Pizza Hut,214,25.305640,55.374543,1.0,10721.046330,0.825014
4,Domino's Pizza,1,30.892860,75.822199,1.0,11049.713548,0.819650
5,Domino's Pizza,1,28.734810,77.114088,1.0,11317.481092,0.815280
6,Jeet Pizza,1,28.712231,77.141626,1.0,11320.732004,0.815227
7,Domino's Pizza,1,28.701408,77.116785,1.0,11321.076735,0.815221
8,Domino's Pizza,1,28.706403,77.162671,1.0,11322.015236,0.815206
9,Domino's Pizza,1,28.699259,77.139456,1.0,11322.025926,0.815205


In [52]:
b

700        0.000000
9183    1100.313921
744      431.217942
2471     276.047364
2483     273.850511
5476     993.335111
5967     991.043268
6838     983.839065
9201     303.290141
819     1000.246853
924      968.292613
Name: 700, dtype: float64